In [1]:
import numpy as np
import pandas as pd
import random
from fastai.tabular import *  # Quick accesss to tabular functionality

In [2]:
import fastai

In [3]:
df = pd.read_csv('../data/train.csv',nrows=80000000)

In [4]:
df.columns

Index(['acoustic_data', 'time_to_failure'], dtype='object')

In [5]:
df_slice_x = [df[i:(150000+i)]['acoustic_data'].values for i in range(0,80000000,150000)][:-1]
df_slice_y = [df[(150000+i):(150000+i+1)]['time_to_failure'].values for i in range(0,80000000,150000)][:-1]

In [6]:
x_mat = np.array(df_slice_x)

In [7]:
y_vec = np.array(df_slice_y)

In [8]:
x_mat.shape, y_vec.shape

((533, 150000), (533, 1))

In [9]:
x_df = pd.DataFrame(x_mat)

In [10]:
y_ser = pd.DataFrame(y_vec)

In [11]:
full_df = pd.concat([x_df,y_ser],axis=1)

In [12]:
full_df.columns = range(0,len(full_df.columns))

In [14]:
full_df.columns

RangeIndex(start=0, stop=150001, step=1)

In [ ]:
doc(TabularDataBunch.from_df)

In [18]:
len_test = int(len(full_df)*.2)
dep_var = [150000]
cont_names = list(full_df)[:-1]
procs = [FillMissing, Categorify, Normalize]
test = TabularList.from_df(full_df.iloc[:len_test].copy(), path='../data/',cont_names=cont_names)

In [19]:
full_df.head()

,0,1,2,3,4,5,6,7,8,9,...,149991,149992,149993,149994,149995,149996,149997,149998,149999,150000
0,12,6,8,5,8,8,9,7,-5,3,...,7,4,7,-2,1,6,6,2,0,1.430797
1,5,6,8,6,3,-1,5,4,4,4,...,2,2,4,5,5,7,7,7,5,1.391499
2,5,5,8,9,9,10,11,12,13,5,...,7,9,8,13,15,6,5,8,3,1.353196
3,5,-5,-4,1,3,4,6,12,15,17,...,6,6,3,10,11,7,3,8,5,1.313798
4,12,6,4,-1,0,6,7,6,2,-2,...,0,-4,-1,2,3,0,6,5,11,1.274400


In [ ]:
data = (TabularList.from_df(full_df, path='../data/',cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(0,len_test)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [ ]:
data.show_batch(rows=10)

In [ ]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [ ]:
learn.fit(1, 1e-2)

In [ ]:
row = full_df.iloc[0]

In [ ]:
learn.predict(row)

# PyTorch

In [ ]:
xi = df_slice_x[0]
yi = df_slice_y[0]

In [ ]:
pd.concat([xi,yi])

In [ ]:
xdf.head()

In [ ]:
xdf = pd.DataFrame(columns=range(0,150000))
for i, (a,b) in enumerate(zip(df_slice_x,df_slice_y)):
    xdf.loc[i] = pd.concat([a,b])

In [ ]:
xandy = np.array([pd.concat([a,b]).values for i,(a,b) in enumerate(zip(df_slice_x,df_slice_y))] )

In [ ]:
xandy.shape

In [ ]:
def readhd5(f1, start, stop):
    df = pd.read_hdf(f1, start=start,stop=stop, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
#     df = store.select('df',start=start,stop=stop)
    return df

df = readhd5(f1="train.hd5", start=700, stop=30000)
df


In [ ]:
data =  pd.read_hd5('train.h5', sep=",", chunksize=chunksize, iterator=True, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32}, low_memory=False)
i = 0 
for df in data:

Lets play pytorch...

In [ ]:
df = pd.read_csv('../data/train.csv',nrows=80000000)

In [ ]:
df_slice_x = [df[i:(150000+i)]['acoustic_data'].astype(np.float32).values for i in range(0,80000000,150000)][:-1]
df_slice_y = [df[(150000+i):(150000+i+1)]['time_to_failure'].astype(np.float32).values for i in range(0,80000000,150000)][:-1]

In [ ]:
x = torch.randn(N, D_in)


In [ ]:
x = torch.tensor(np.array(df_slice_x),dtype=torch.float32)
y = torch.tensor(np.array(df_slice_y).reshape((-1,1)),dtype=torch.float32)

In [ ]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 150000, 100, 1

# Create random Tensors to hold inputs and outputs
#x = torch.randn(N, D_in)
#y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

In [ ]:
model(x[0])

In [ ]:
x[0].shape